In [1]:
#check gpu usage
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Apr 14 06:19:28 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 474.64       Driver Version: 474.64       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 N/A |                  N/A |
| 30%    0C    P0    N/A /  N/A |    262MiB /  4096MiB |     N/A      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#check ram usage
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 17.1 gigabytes of available RAM

Not using a high-RAM runtime


In [3]:
!pip install datasets
!pip install transformers
!pip3 install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu102/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu102/torch_stable.html


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from huggingface_hub import notebook_login
from datasets import load_dataset
import tensorflow as tf
import pandas as pd
import numpy as np
import torch

In [8]:
#Reading untouched data (non-preprocessed, non-augmented)

eng_tam_train_nonprepro_nonaug_dataset = load_dataset("csv", data_files=r"D:\Files\mini prj\eng_tam_train_nonprepro_nonaug.csv")
eng_tam_train_nonprepro_nonaug = eng_tam_train_nonprepro_nonaug_dataset["train"]

eng_tam_test_nonprepro_nonaug_dataset = load_dataset("csv", data_files=r"D:\Files\mini prj\eng_tam_test_nonprepro_nonaug.csv")
eng_tam_test_nonprepro_nonaug = eng_tam_test_nonprepro_nonaug_dataset["train"]

print(len(eng_tam_train_nonprepro_nonaug), len(eng_tam_test_nonprepro_nonaug),)

3861 1207


In [9]:
model_ckpt = "bert-base-multilingual-cased"

In [10]:

from transformers import AutoTokenizer

model_ckpt = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

"""
from transformers import BertTokenizer

model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
"""

'\nfrom transformers import BertTokenizer\n\nmodel_name = "bert-base-multilingual-cased"\ntokenizer = BertTokenizer.from_pretrained(model_name)\n'

In [11]:
# Define tokenizer function

def tokenize(batch):
  return tokenizer(batch["text"], padding=True, truncation=True)

In [12]:
# Ensuring there are no empty records so that tokenization doesn't fail

eng_tam_train_nonprepro_nonaug = eng_tam_train_nonprepro_nonaug.filter(lambda example: example['text'] is not None)
eng_tam_test_nonprepro_nonaug = eng_tam_test_nonprepro_nonaug.filter(lambda example: example['text'] is not None)


print( len(eng_tam_train_nonprepro_nonaug), len(eng_tam_test_nonprepro_nonaug))

Filter:   0%|          | 0/3861 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1207 [00:00<?, ? examples/s]

3861 1207


In [13]:
# Apply tokenizer across all splits in the corpus

eng_tam_train_nonprepro_nonaug_encoded = eng_tam_train_nonprepro_nonaug.map(tokenize, batched=True, batch_size=None)
eng_tam_test_nonprepro_nonaug_encoded = eng_tam_test_nonprepro_nonaug.map(tokenize, batched=True, batch_size=None)


Map:   0%|          | 0/3861 [00:00<?, ? examples/s]

Map:   0%|          | 0/1207 [00:00<?, ? examples/s]

In [14]:
print(eng_tam_test_nonprepro_nonaug_encoded.column_names)

['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask']


In [15]:
!pip install scikit-learn


In [16]:
!pip install accelerate>=0.21.0


In [17]:
#Define performance metrics

from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average = "macro")
  acc = accuracy_score(labels, preds)
  return {"accuracy": acc, "f1": f1}

In [19]:
#Extract labels separately

Y_eng_tam_test_nonprepro_nonaug = np.array(eng_tam_test_nonprepro_nonaug['label'])


In [25]:
#Tamil English

In [20]:
from transformers import AutoModelForSequenceClassification

num_labels = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
eng_tam_model_base_prepro = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, 
                                                            num_labels=num_labels)
.to(device))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
#Define hyperparameters

from transformers import Trainer, TrainingArguments

batch_size = 8
logging_steps = len(eng_tam_test_nonprepro_nonaug_encoded) // batch_size
model_name = f"{model_ckpt}-eng_tam_model_base_prepro"
eng_tam_training_args_base_prepro = TrainingArguments(
    output_dir = model_name,
    num_train_epochs = 5,
    learning_rate = 3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    disable_tqdm = False,
    logging_steps =logging_steps,
    log_level = "error"
)

In [23]:
trainer = Trainer(
    model = eng_tam_model_base_prepro,
    args = eng_tam_training_args_base_prepro,
    compute_metrics = compute_metrics,
    train_dataset = eng_tam_train_nonprepro_nonaug_encoded,
    eval_dataset = eng_tam_test_nonprepro_nonaug_encoded,
    tokenizer = tokenizer
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.428100,0.418882,0.898923,0.315590
2,0.414600,0.419995,0.898923,0.315590
3,0.415200,0.398271,0.898923,0.315590
4,0.407900,0.384603,0.898923,0.315590
5,0.334100,0.337044,0.899751,0.349039


TrainOutput(global_step=2415, training_loss=0.40990241645038994, metrics={'train_runtime': 29909.7317, 'train_samples_per_second': 0.645, 'train_steps_per_second': 0.081, 'total_flos': 4593289642651710.0, 'train_loss': 0.40990241645038994, 'epoch': 5.0})

In [24]:
eng_tam_preds_output_base_nonprepro = trainer.predict(eng_tam_test_nonprepro_nonaug_encoded)

In [25]:
eng_tam_y_preds_base_nonprepro = np.argmax(eng_tam_preds_output_base_nonprepro.predictions, axis=1)

In [26]:
from sklearn.metrics import classification_report

target_names = ['homophobic', 'Non-anti LGBTQIA+ content', 'transphobic']
print(classification_report(Y_eng_tam_test_nonprepro_nonaug, eng_tam_y_preds_base_nonprepro, target_names=target_names))

                           precision    recall  f1-score   support

               homophobic       0.36      0.06      0.10        88
Non-anti LGBTQIA+ content       0.91      1.00      0.95      1085
              transphobic       0.00      0.00      0.00        34

                 accuracy                           0.90      1207
                macro avg       0.42      0.35      0.35      1207
             weighted avg       0.84      0.90      0.86      1207



C:\Users\AKC\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\AKC\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\AKC\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier